**Analiza współczynnika uwarunkowania macierzy**\
Łukasz Jarzęcki 331697

Kiedy macierz jest źle uwarunkowana:
    - kiedy wartości własne są małe w porównaniu z największą wartością własną
    - kiedy macierzy blisko jest do bycia macierzą osobliwą

Poprawianie współczynnika uwarunkowania:
    - skalowanie macierzy
    - regularizacja - dodanie małej wartości na przekątnej
    - prekondycjonowanie
    - wybór odpowiedniej metody numerycznej

In [1]:
import numpy as np
import re
from scipy.io import loadmat
import time
import matplotlib.pyplot as plt

1. **Funkcje pomocnicze**

In [3]:
def load_A_b_from_m_file(filename):
    with open(filename, 'r') as f:
        content = f.read()

    content = re.sub(r'%.*', '', content)

    A_match = re.search(r'A\s*=\s*\[(.*?)\];', content, re.DOTALL)
    if not A_match:
        raise ValueError("Nie znaleziono macierzy A")
    A_rows = A_match.group(1).strip().split('\n')
    A = [list(map(float, row.strip().split())) for row in A_rows]

    b_match = re.search(r'b\s*=\s*\[(.*?)\](\'?);', content, re.DOTALL)
    if not b_match:
        raise ValueError("Nie znaleziono wektora b")
    b_data = b_match.group(1).strip().replace('\n', ' ').split()
    b = np.array(list(map(float, b_data)))
    if b_match.group(2) == "'":  # był apostrof, więc wektor pionowy
        b = b.reshape(-1, 1)
    b = b.flatten()
    return np.array(A), b

In [2]:
def load_A_b_from_mat_file(filename):
    data = loadmat(filename)  # ścieżka do Twojego pliku .mat
    A = data['A']
    b = data['b']
    return A,b.flatten()

In [4]:
def generate_matrices(n):

    A = np.random.uniform(-1, 1, (n, n))
    for i in range(n):
        A[i, i] = np.sum(np.abs(A[i])) + 1  # zagwarantuj dominację przekątnej

    b = np.random.uniform(-10, 10, n)

    return A, b

In [62]:
def generate_spd(n):
    A = np.random.rand(n,n)

    return A.T@A

2. **Poprawianie współczynnika uwarunkowania**

Poprzez skalowanie

In [ ]:
A,b = load_A_b_from_mat_file("pink.mat") #pink - macierz źle uwarunkowana

In [23]:
print(f"przed skalowaniem: {np.linalg.cond(A)}")
 
S = np.diag(1/np.sqrt(np.diag(A)))
A_scaled = S*A*S

print(f"po skalowaniu: {np.linalg.cond(A_scaled)}")


przed skalowaniem: 1844.6339859461466
po skalowaniu: 1.0000000000000007


3. **Wyznaczanie współczynnika uwarunkowania metodą iteracyjną**

Implementacja metody

In [ ]:
def calc_max_sing_val(A):
    stop = 10e-6

    x = np.ones(A.shape[0])
    x /= np.linalg.norm(x)

    lambda_old = 0
    

    while(abs(lambda_old-lambda_new)/lambda_old)>stop):
        x_next = A@x
        x_next = x_next/np.linalg.norm(x_next)

        lambda_new = x_next.T @ A @ x_next

        x = x_next

    return lambda_new
        

In [65]:
A = generate_spd(2)
A

array([[0.03765903, 0.1344182 ],
       [0.1344182 , 0.4800045 ]])

In [82]:
A = np.array([
    [22, 21, -8, 4],
    [21, 49, 11, 39],
    [-8, 11, 60, -19],
    [4, 39, -19, 135]
])

print(calc_max_sing_val(A))

1.0000041688335264


/tmp/ipykernel_12564/1543985685.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  while(abs(lambda_old-lambda_new)/abs(lambda_old)>stop):
